# 1. How to read real sequences from online database
## 1.1. Useful libraries

In [1]:
using FastaIO
using OffsetArrays
using PyPlot
using DelimitedFiles
using BenchmarkTools
using StatsBase
using LinearAlgebra
using Printf
using HTTP

## 1.2. Read sequences from online database

In [73]:
function sequenceDownload(sequence)

    sequenceFile = sequence * ".fasta"

    URL = "https://www.uniprot.org/uniprotkb/" * sequenceFile

    query = HTTP.get(URL)
    fastaString=String(query.body)

    open(sequenceFile,"w") do f
        write(f,fastaString)
    end

    FastaIO.readfasta(sequenceFile)[1][2]
end

sequenceDownload (generic function with 1 method)

## 1.3. An example: loading some real biological sequences

In [85]:
HBB_Human = sequenceDownload("P68871")
HBA_Bonobo = sequenceDownload("P69906")
HBA_Chimp = sequenceDownload("P69907")

"MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"

# 2. Distances between two sequences
## 2.1 Hamming distance
The [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance) between two sequences $X=(x_1,\dots,x_n)$ and $Y=(y_1,\dots,y_m)$ is computed as the minimum number of substitutions needed to transform the second sequence into the first one. This type of distance can be computed only between two same length sequences. For example:

- X = `"GGGAATTTCC"`
- Y = `"GGCAATAACC"`
-        "vvxvvvxxvv"

("v" for a *match* and "x" for a *substitution*) then the Hamming distance here is $d_H(X,Y)=3$.

In [102]:
function hamming(x,y)
    if length(x) != length(y)
        println("ERROR: sequences should have equal lenghts!")
    else
        return sum(x[i] != y[i] for i in eachindex(x))
    end
end

hamming (generic function with 1 method)

In [106]:
# From previous exmple:
X, Y = "GGGAATTTCC", "GGCAATAACC"
hamming(X,Y)

3

## 2.2 Levenshtein distance 
The [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance) between two sequences $X=(x_1,\dots,x_n)$ and $Y=(y_1,\dots,y_m)$ is computed as the minimum number of substitutions, insertions or deletions (reflecting the three possible biological operations, namely *mutations*) needed to transform the second sequence into the first one. This type of distance can be computed between two any length sequences. For example:

- X_2 = `"GG-AATTTCC"`
- Y_2 = `"GGCAAT--AC"`

then the Levenshtein distance here is $d_L(X,Y)=4$.

In [109]:
function levenshtein(x,y)
    # Recursive approach: dynamic programming using a dictionary
    D = Dict()

    function lev(x,y)
        isempty(x) && return length(y) # Delete all elements in 2nd sequence
        isempty(y) && return length(x) # Insert all elements in 2nd sequence

        haskey(D,(x,y)) && return D[(x,y)]
        
        D[(x,y)] = min(1 - (x[end] == y[end]) + lev(x[1:end-1],y[1:end-1]), 1 + lev(x[1:end-1],y), 1 + lev(x,y[1:end-1]))
    end

    lev(x,y)
end

levenshtein (generic function with 1 method)

In [111]:
# From previous exmple:
X2, Y2 = "GGAATTTCC", "GGCAATAC"
levenshtein(X2,Y2)

4

## 2.3 An example with loaded real biological sequences

In [114]:
@show length(HBB_Human), length(HBA_Bonobo), length(HBA_Chimp);
@show hamming(HBB_Human, HBA_Bonobo);
@show hamming(HBA_Bonobo, HBA_Chimp);
@show levenshtein(HBB_Human, HBA_Bonobo);
@show levenshtein(HBA_Bonobo, HBA_Chimp);
@show levenshtein(HBB_Human, HBA_Chimp);

(length(HBB_Human), length(HBA_Bonobo), length(HBA_Chimp)) = (147, 142, 142)
ERROR: sequences should have equal lenghts!
hamming(HBB_Human, HBA_Bonobo) = nothing
hamming(HBA_Bonobo, HBA_Chimp) = 0
levenshtein(HBB_Human, HBA_Bonobo) = 84
levenshtein(HBA_Bonobo, HBA_Chimp) = 0
levenshtein(HBB_Human, HBA_Chimp) = 84


# 3. [Alignment algorithms](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm#Scoring_systems)
## 3.1. Global, simple (Levenshtein) alignment
Global, simple alignment between $X=(x_1,\dots,x_n)$ and $Y=(y_1,\dots,y_m)$.
- Initialization: $L(0,0)=0,\ L(i,0)=i,\ L(0,j)=j\ \forall\ i,j$

- Recursion: $L(i,j)=\min{\begin{cases}1 - \delta(x_i,y_j) + L(i-1,j-1) & \text{Substitution}\\1+L(i-1,j) & \text{Deletion (in X)}\\1+L(i,j-1) & \text{Insertion (in Y)}\end{cases}}$ for $\begin{cases}0\leq i \leq n \\ 0\leq j \leq m\end{cases}$

- Termination: $L(n,m)$ optimal score

In [70]:
function simpleAlignment(x,y)
    
    L = OffsetArray(zeros(length(x) + 1, length(y) + 1), 0:length(x), 0:length(y)) # Distances matrix
    B = OffsetArray(zeros(length(x) + 1, length(y) + 1), 0:length(x), 0:length(y)) # Operations matrix: 1 -> substitution; 2 -> deletion; 3 -> insertion

    for i in eachindex(x)
        L[i,0] = i 
        B[i,0] = 2 # Gap on top sequence (deletion)
        for j in eachindex(y)
            L[0,j] = j
            B[0,j] = 3 # Gap on left sequence (insertion)
            L[i,j], B[i,j] = findmin([1 - (x[i] == y[j]) + L[i-1,j-1], 1 + L[i-1,j], 1 + L[i,j-1]]) # findmin(v=[A, B, C]) returns the minimum value and its "position" in vector v = [A, B, C]; "1" corresponds to substitution,  "2" corresponds to deletion,  "3" corresponds to insertion
        end
    end
    return L, B
end

simpleAlignment (generic function with 1 method)

In [123]:
@show length(HBB_Human), length(HBA_Bonobo), length(HBA_Chimp);
@show hamming(HBB_Human, HBA_Bonobo), levenshtein(HBB_Human, HBA_Bonobo);
@show hamming(HBA_Bonobo, HBA_Chimp), levenshtein(HBA_Bonobo, HBA_Chimp);
@show simpleAlignment(HBB_Human, HBA_Bonobo)[1][end:end], levenshtein(HBB_Human, HBA_Bonobo);
@show simpleAlignment(HBB_Human, HBA_Chimp)[1][end:end], levenshtein(HBB_Human, HBA_Chimp);
@show simpleAlignment(HBA_Bonobo, HBA_Chimp)[1][end:end], levenshtein(HBA_Bonobo, HBA_Chimp);

(length(HBB_Human), length(HBA_Bonobo), length(HBA_Chimp)) = (147, 142, 142)
ERROR: sequences should have equal lenghts!
(hamming(HBB_Human, HBA_Bonobo), levenshtein(HBB_Human, HBA_Bonobo)) = (nothing, 84)
(hamming(HBA_Bonobo, HBA_Chimp), levenshtein(HBA_Bonobo, HBA_Chimp)) = (0, 0)
(((simpleAlignment(HBB_Human, HBA_Bonobo))[1])[end:end], levenshtein(HBB_Human, HBA_Bonobo)) = ([84.0], 84)
(((simpleAlignment(HBB_Human, HBA_Chimp))[1])[end:end], levenshtein(HBB_Human, HBA_Chimp)) = ([84.0], 84)
(((simpleAlignment(HBA_Bonobo, HBA_Chimp))[1])[end:end], levenshtein(HBA_Bonobo, HBA_Chimp)) = ([0.0], 0)


## 3.2. Scoring scheme
### Substitution matrix
$$S(X,Y)=\sum_{i=1}^N\log{\frac{p_{x_iy_i}}{q_{x_i}q_{y_i}}}=\sum_{i=1}^Ns(x_i,y_i)$$
### Gap score
$$\begin{cases}\gamma(g)=-dg\\\gamma(g)=-d-e(g-1) & e < d\end{cases}$$

Score $s(x_i,y_j)$ is taken from [BLOSUM50](https://www.ncbi.nlm.nih.gov/IEB/ToolBox/C_DOC/lxr/source/data/BLOSUM50).

In [22]:
# Entries for the BLOSUM50 matrix at a scale of ln(2)/3.0.
# https://www.ncbi.nlm.nih.gov/IEB/ToolBox/C_DOC/lxr/source/data/BLOSUM50
#  A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  J  Z  X  *

blosum50 = [
[  5 -2 -1 -2 -1 -1 -1  0 -2 -1 -2 -1 -1 -3 -1  1  0 -3 -2  0 -2 -2 -1 -1 -5 ]
[ -2  7 -1 -2 -4  1  0 -3  0 -4 -3  3 -2 -3 -3 -1 -1 -3 -1 -3 -1 -3  0 -1 -5 ]
[ -1 -1  7  2 -2  0  0  0  1 -3 -4  0 -2 -4 -2  1  0 -4 -2 -3  5 -4  0 -1 -5 ]
[ -2 -2  2  8 -4  0  2 -1 -1 -4 -4 -1 -4 -5 -1  0 -1 -5 -3 -4  6 -4  1 -1 -5 ]
[ -1 -4 -2 -4 13 -3 -3 -3 -3 -2 -2 -3 -2 -2 -4 -1 -1 -5 -3 -1 -3 -2 -3 -1 -5 ]
[ -1  1  0  0 -3  7  2 -2  1 -3 -2  2  0 -4 -1  0 -1 -1 -1 -3  0 -3  4 -1 -5 ]
[ -1  0  0  2 -3  2  6 -3  0 -4 -3  1 -2 -3 -1 -1 -1 -3 -2 -3  1 -3  5 -1 -5 ]
[  0 -3  0 -1 -3 -2 -3  8 -2 -4 -4 -2 -3 -4 -2  0 -2 -3 -3 -4 -1 -4 -2 -1 -5 ]
[ -2  0  1 -1 -3  1  0 -2 10 -4 -3  0 -1 -1 -2 -1 -2 -3  2 -4  0 -3  0 -1 -5 ]
[ -1 -4 -3 -4 -2 -3 -4 -4 -4  5  2 -3  2  0 -3 -3 -1 -3 -1  4 -4  4 -3 -1 -5 ]
[ -2 -3 -4 -4 -2 -2 -3 -4 -3  2  5 -3  3  1 -4 -3 -1 -2 -1  1 -4  4 -3 -1 -5 ]
[ -1  3  0 -1 -3  2  1 -2  0 -3 -3  6 -2 -4 -1  0 -1 -3 -2 -3  0 -3  1 -1 -5 ]
[ -1 -2 -2 -4 -2  0 -2 -3 -1  2  3 -2  7  0 -3 -2 -1 -1  0  1 -3  2 -1 -1 -5 ]
[ -3 -3 -4 -5 -2 -4 -3 -4 -1  0  1 -4  0  8 -4 -3 -2  1  4 -1 -4  1 -4 -1 -5 ]
[ -1 -3 -2 -1 -4 -1 -1 -2 -2 -3 -4 -1 -3 -4 10 -1 -1 -4 -3 -3 -2 -3 -1 -1 -5 ]
[  1 -1  1  0 -1  0 -1  0 -1 -3 -3  0 -2 -3 -1  5  2 -4 -2 -2  0 -3  0 -1 -5 ]
[  0 -1  0 -1 -1 -1 -1 -2 -2 -1 -1 -1 -1 -2 -1  2  5 -3 -2  0  0 -1 -1 -1 -5 ]
[ -3 -3 -4 -5 -5 -1 -3 -3 -3 -3 -2 -3 -1  1 -4 -4 -3 15  2 -3 -5 -2 -2 -1 -5 ]
[ -2 -1 -2 -3 -3 -1 -2 -3  2 -1 -1 -2  0  4 -3 -2 -2  2  8 -1 -3 -1 -2 -1 -5 ]
[  0 -3 -3 -4 -1 -3 -3 -4 -4  4  1 -3  1 -1 -3 -2  0 -3 -1  5 -3  2 -3 -1 -5 ]
[ -2 -1  5  6 -3  0  1 -1  0 -4 -4  0 -3 -4 -2  0  0 -5 -3 -3  6 -4  1 -1 -5 ]
[ -2 -3 -4 -4 -2 -3 -3 -4 -3  4  4 -3  2  1 -3 -3 -1 -2 -1  2 -4  4 -3 -1 -5 ]
[ -1  0  0  1 -3  4  5 -2  0 -3 -3  1 -1 -4 -1  0 -1 -2 -2 -3  1 -3  5 -1 -5 ]
[ -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -5 ]
[ -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5  1 ]
]

# aal = ["A" "R" "N" "D" "C" "Q" "E" "G" "H" "I" "L" "K" "M" "F" "P" "S" "T" "W" "Y" "V" "B" "J" "Z" "X" "*"]
# aa = Dict(aal[k] => k for k in 1:length(aal))

function blosum(x,y)
    
    aal = ["A" "R" "N" "D" "C" "Q" "E" "G" "H" "I" "L" "K" "M" "F" "P" "S" "T" "W" "Y" "V" "B" "J" "Z" "X" "*"]
    aa = Dict(aal[k] => k for k in 1:length(aal))
    
    return blosum50[aa[string(x)],aa[string(y)]]
end

blosum (generic function with 1 method)

## 3.3. Global, scoring scheme & linear gap penalties alignment
Global, scoring scheme & linear gap penalties alignment between $X=(x_1,\dots,x_n)$ and $Y=(y_1,\dots,y_m)$.
- Initialization: $F(0,0)=0,\ F(i,0)=-id,\ F(0,j)=-jd\ \forall\ i,j$

- Recursion: $F(i,j)=\max{\begin{cases}L(i-1,j-1) + s(x_i,y_j)& \text{Substitution}\\L(i-1,j) - d & \text{Deletion (in X)}\\L(i,j-1) - d & \text{Insertion (in Y)}\end{cases}}$ for $\begin{cases}0\leq i \leq n \\ 0\leq j \leq m\end{cases}$

- Termination: $F(n,m)$ optimal score

In [69]:
function globalAlignment(x,y)
    
    d = 8 # Linear gap penalty
    F = OffsetArray(zeros(length(x) + 1, length(y) + 1), 0:length(x), 0:length(y)) # Scores matrix
    B = OffsetArray(zeros(length(x) + 1, length(y) + 1), 0:length(x), 0:length(y)) # Operations matrix: 1 -> substitution; 2 -> deletion; 3 -> insertion

    for i in eachindex(x)
        F[i,0] = - d * i # Score (cost) for deleting all elements
        B[i,0] = 2 # Gap on top sequence (deletion)
        for j in eachindex(y)
            F[0,j] = - d * j # Score (cost) for inserting all elements
            B[0,j] = 3 # Gap on left sequence (insertion)
            F[i,j], B[i,j] = findmax([F[i,j] + blosum(x[i],y[j]), F[i-1,j] - d, F[i,j-1] - d]) # findmin(v=[A, B, C]) returns the minimum value and its "position" in vector v = [A, B, C]; "1" corresponds to substitution, "2" corresponds to deletion, "3" corresponds to insertion
        end
    end
    return F, B
end

globalAlignment (generic function with 1 method)

In [122]:
@show length(HBB_Human), length(HBA_Bonobo), length(HBA_Chimp);
@show hamming(HBB_Human, HBA_Bonobo), levenshtein(HBB_Human, HBA_Bonobo);
@show hamming(HBA_Bonobo, HBA_Chimp), levenshtein(HBA_Bonobo, HBA_Chimp);
@show globalAlignment(HBB_Human, HBA_Bonobo)[1][end:end], levenshtein(HBB_Human, HBA_Bonobo);
@show globalAlignment(HBB_Human, HBA_Chimp)[1][end:end], levenshtein(HBB_Human, HBA_Chimp);
@show globalAlignment(HBA_Bonobo, HBA_Chimp)[1][end:end], levenshtein(HBA_Bonobo, HBA_Chimp);

(length(HBB_Human), length(HBA_Bonobo), length(HBA_Chimp)) = (147, 142, 142)
ERROR: sequences should have equal lenghts!
(hamming(HBB_Human, HBA_Bonobo), levenshtein(HBB_Human, HBA_Bonobo)) = (nothing, 84)
(hamming(HBA_Bonobo, HBA_Chimp), levenshtein(HBA_Bonobo, HBA_Chimp)) = (0, 0)
(((globalAlignment(HBB_Human, HBA_Bonobo))[1])[end:end], levenshtein(HBB_Human, HBA_Bonobo)) = ([0.0], 84)
(((globalAlignment(HBB_Human, HBA_Chimp))[1])[end:end], levenshtein(HBB_Human, HBA_Chimp)) = ([0.0], 84)
(((globalAlignment(HBA_Bonobo, HBA_Chimp))[1])[end:end], levenshtein(HBA_Bonobo, HBA_Chimp)) = ([7.0], 0)


## 3.4. Local, scoring scheme & linear gap penalties alignment
Local, scoring scheme & linear gap penalties alignment between $X=(x_1,\dots,x_n)$ and $Y=(y_1,\dots,y_m)$.
- Initialization: $F(0,0)=0,\ F(i,0)=0,\ F(0,j)=0\ \forall\ i,j$

- Recursion: $F(i,j)=\max{\begin{cases}L(i-1,j-1) + s(x_i,y_j)& \text{Substitution}\\L(i-1,j) - d & \text{Deletion (in X)}\\L(i,j-1) - d & \text{Insertion (in Y)}\\ 0 & \text{start new local alignment}\end{cases}}$ for $\begin{cases}0\leq i \leq n \\ 0\leq j \leq m\end{cases}$

- Termination: $\max_{n,m}\{F\}$ optimal score ($F$ is the score matrix)

In [127]:
function localAlignment(x,y)
    
    d = 8 # Linear gap penalty
    F = OffsetArray(zeros(length(x) + 1, length(y) + 1), 0:length(x), 0:length(y)) # Scores matrix
    B = OffsetArray(zeros(length(x) + 1, length(y) + 1), 0:length(x), 0:length(y)) # Operations matrix: 1 -> substitution; 2 -> deletion; 3 -> insertion; 4 -> start new local alignment

    for i in eachindex(x)
        F[i,0] = 0 # Score (cost) for deleting all elements (0, since it will be started a new local alignment)
        B[i,0] = 2 # Gap on top sequence (deletion)
        for j in eachindex(y)
            F[0,j] = 0 # Score (cost) for inserting all elements (0, since it will be started a new local alignment)
            B[0,j] = 3 # Gap on left sequence (insertion)
            F[i,j], B[i,j] = findmax([F[i,j] + blosum(x[i],y[j]), F[i-1,j] - d, F[i,j-1] - d, 0]) # findmin(v=[A, B, C, D]) returns the minimum value and its "position" in vector v = [A, B, C, D]; "1" corresponds to substitution, "2" corresponds to deletion, "3" corresponds to insertion, "4" corresponds to start a new local alignment
        end
    end
    return F, B
end

localAlignment (generic function with 1 method)

## 3.5. Overlap, scoring scheme & linear gap penalties
Overlap, scoring scheme & linear gap penalties alignment between $X=(x_1,\dots,x_n)$ and $Y=(y_1,\dots,y_m)$.
- Initialization: $F(0,0)=0,\ F(i,0)=0,\ F(0,j)=0\ \forall\ i,j$

- Recursion: $F(i,j)=\max{\begin{cases}L(i-1,j-1) + s(x_i,y_j)& \text{Substitution}\\L(i-1,j) - d & \text{Deletion (in X)}\\L(i,j-1) - d & \text{Insertion (in Y)}\end{cases}}$ for $\begin{cases}0\leq i \leq n \\ 0\leq j \leq m\end{cases}$

- Termination: $F(n,m)$ optimal score ($F$ is the score matrix)

In [126]:
function OverlapAlignment1(x,y)
    
    d = 8 # Linear gap penalty
    F = OffsetArray(zeros(length(x) + 1, length(y) + 1), 0:length(x), 0:length(y)) # Scores matrix
    B = OffsetArray(zeros(length(x) + 1, length(y) + 1), 0:length(x), 0:length(y)) # Operations matrix: 1 -> substitution; 2 -> deletion; 3 -> insertion

    for i in eachindex(x)
        F[i,0] = 0
        B[i,0] = 2 # Gap on top sequence (deletion)
        for j in eachindex(y)
            F[0,j] = 0 
            B[0,j] = 3 # Gap on left sequence (insertion)
            F[i,j], B[i,j] = findmax([F[i,j] + blosum(x[i],y[j]), F[i-1,j] - d, F[i,j-1] - d]) # findmin(v=[A, B, C]) returns the minimum value and its "position" in vector v = [A, B, C]; "1" corresponds to substitution, "2" corresponds to deletion, "3" corresponds to insertion
        end
    end
    return F, B
end

OverlapAlignment1 (generic function with 1 method)

# 3.6 Alignment (full)

In [128]:
function score(x,y;method)
    
    if method == "simple"
        d = 1 # Gap penalty = operation number (1 each time)
        cost = simplecost
    elseif method == "global" || method == "local" || method == "overlap"
        d = 8 # Linear gap penalty
        cost = blosum
    end

    n, m = length(x), length(y)
    
    F = OffsetArray(zeros(n + 1, m + 1), 0:n, 0:m) # Scores matrix
    B = OffsetArray(zeros(n + 1, m + 1), 0:n, 0:m) # Operations matrix: 1 -> substitution; 2 -> deletion; 3 -> insertion
    
    if method == "local" || method == "overlap"
        F[0:n,0] .= 0 # Score (cost) for deleting all elements (0, since it will be started a new local alignment)
        F[0,0:m] .= 0 # Score (cost) for inserting all elements (0, since it will be started a new local alignment)
    else
        F[0:n,0] .= (0:n) * (-d) # Score (cost) for deleting all elements
        F[0,0:m] .= (0:m) * (-d) # Score (cost) for inserting all elements
    end
    
    B[0:n,0] .= 2 # Gaps on top sequence (deletions)
    B[0,0:m] .= 3 # Gaps on left sequence (insertions)
    
    for i = 1:n
        for j = 1:m
            if method == "local"
                F[i,j], B[i,j] = findmax([F[i-1,j-1] + cost(x[i],y[j]), F[i-1,j] - d , F[i,j-1] - d, 0]) # findmin(v=[A, B, C, D]) returns the minimum value and its "position" in vector v = [A, B, C, D]; "1" corresponds to substitution, "2" corresponds to deletion, "3" corresponds to insertion, "4" corresponds to start a new local alignment
            else
                F[i,j], B[i,j] = findmax([F[i-1,j-1] + cost(x[i],y[j]), F[i-1,j] - d , F[i,j-1] - d]) # findmin(v=[A, B, C]) returns the minimum value and its "position" in vector v = [A, B, C]; "1" corresponds to substitution, "2" corresponds to deletion, "3" corresponds to insertion
            end
        end
    end
    
    return F, B
    
end

function simplecost(a,b)
    -(a!=b) 
 end

simplecost (generic function with 1 method)

In [ ]:
function traceback(x)
    if     x == 1 return "↖" #substitution
    elseif x == 2 return "↑" #deletion
    elseif x == 3 return "←" #insertion
    elseif x == 4 return " "
    end
end